In [ ]:
import os
from __future__ import print_function
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

In [ ]:
data_path = ['./dados/']
filepath = os.sep.join(data_path + ['economy.csv'])
data = pd.read_csv(filepath, sep=',')
data_orig = data.copy()
data2 = data.copy()

filepath = os.sep.join(data_path + ['results.csv'])
results = pd.read_csv(filepath, sep=',')

In [ ]:
def search_match_winer(match_id):
    row = results.loc[results['match_id'] == match_id]
    return row.head(1)['match_winner']

winner =[]
for i,row in data.iterrows():
    match_winner = search_match_winer(row['match_id'])
    winner.append(match_winner.values)

In [ ]:
winner_pd= []
for win in winner:
    if win:
        winner_pd.append(win[0])
    else: 
        winner_pd.append(np.nan)

In [ ]:
print(data.shape)
data['match_winner'] = pd.DataFrame(winner_pd)
print(data.shape)
data.dropna(subset=['match_winner'],inplace=True)
print(data.shape)

In [ ]:
data = data.drop(['date'], axis=1)
data = data.drop(['match_id'], axis=1)
data = data.drop(['event_id'], axis=1)
data = data.drop(['best_of'], axis=1)
data = data.drop(['team_1'], axis=1)
data = data.drop(['team_2'], axis=1)

In [ ]:

data = data.replace(np.nan,0)
data.head(1)

In [ ]:
p = data.hist(figsize = (20,20))

In [ ]:
def oneHotEncoding(df, column):
    one_hot_encoded = pd.get_dummies(df[column])
    df = df.drop(column,axis = 1)
    df = df.join(one_hot_encoded)
    return df

data_aux = {}
pd.DataFrame(data_aux)
categorical_features = ['_map'] # ['_map', 'team_1', 'team_2']
for i in categorical_features:
    data_aux = oneHotEncoding(data, i)
    data= data_aux.copy()

In [ ]:
## estamos subistiuindo para inteiros onde t = o e ct =1 
data['t1_start'] = data['t1_start'].replace('t',0)
data['t1_start'] = data['t1_start'].replace('ct',1)
data['t2_start'] = data['t2_start'].replace('t',0)
data['t2_start'] = data['t2_start'].replace('ct',1)

In [ ]:
for col in data.columns:
    data[col] = data[col].astype(np.float)
data.dtypes

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
data = data.reset_index(drop=True)
X = data.drop('match_winner', axis=1)
y = data.match_winner
sss = StratifiedShuffleSplit(n_splits=10, random_state=50)

def get_avg_score(n):
    pipe = [
        ('scaler', MinMaxScaler()),
        ('pca', PCA(n_components=n)),
        ('estimator', LogisticRegression(max_iter=1000))
    ]
    pipe = Pipeline(pipe)
    scores = []
    
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        pipe.fit(X_train, y_train)
        scores.append(accuracy_score(y_test, pipe.predict(X_test)))
    return np.mean(scores)

In [ ]:

ns = np.arange(1,103)
score_list = [get_avg_score(n) for n in ns]

In [ ]:
sns.set_context('talk')
ax = plt.axes()
ax.plot(ns, score_list)
ax.set(xlabel='Number of Dimensions',
       ylabel='Average Accuracy',
       title='LogisticRegression ACC vs Number of dimensions')
ax.grid(True)
plt.savefig(f'StandardScaler.png', dpi=100)
print(score_list)

[0.5472423573904821, 0.7055783170501103, 0.7073747242357389, 0.710463283958399, 0.7267254963756697, 0.7323983611723921, 0.7370942325874568, 0.7379136463914276, 0.7387015442798612, 0.7398046013236684, 0.7417901040025212, 0.7521273242987708, 0.7520327765521589, 0.7520012606366214, 0.7511818468326505, 0.7515915537346359, 0.7522533879609202, 0.7579892845887172, 0.7582098959974788, 0.7598487236054208, 0.7587141506460762, 0.7606051055783171, 0.7624330286794831, 0.7620863536085725, 0.7745351402458241, 0.7958398991490704, 0.7958714150646076, 0.7965647652064293, 0.796816892530728, 0.7971005357705641, 0.7976678222502364, 0.7991805861960289, 0.8078789788843365, 0.8098959974787266, 0.8098014497321147, 0.8091396155058306, 0.8105263157894738, 0.809643870154428, 0.8100850929719507, 0.8092026473369052, 0.8096753860699654, 0.8107469271982352, 0.8107784431137723, 0.8122281752284903, 0.8125748502994012, 0.8124172707217145, 0.8126693980460132, 0.814276709738418, 0.8155058304443745, 0.8153167349511502, 0.8162937283328082, 0.8154743145288371, 0.8161046328395839, 0.8160416010085093, 0.8165773715726441, 0.8167034352347935, 0.8172392057989285, 0.8174913331232272, 0.817018594390167, 0.8176804286164513, 0.8173022376300032, 0.8203277655215884, 0.820201701859439, 0.820737472423574, 0.8209265679167979, 0.8213047589032462, 0.8210841474944847, 0.8219035612984558, 0.8224393318625906, 0.8225023636936653, 0.8220926567916799, 0.8224393318625906, 0.8229751024267256, 0.8233217774976364, 0.823573904821935, 0.8250866687677277, 0.8265048849669082, 0.8258745666561614, 0.8287109990545225, 0.829026158209896, 0.8295934446895682, 0.8294358651118815, 0.8288685786322093, 0.8299401197604791, 0.8299716356760163, 0.8304128584935393, 0.8313268200441224, 0.8316419791994958, 0.8314213677907343, 0.8380081941380398, 0.8383548692089505, 0.8381657737157264, 0.838575480617712, 0.8382603214623383, 0.838670028364324, 0.8385439647021746, 0.838670028364324, 0.838670028364324, 0.8386385124487866, 0.838575480617712, 0.838575480617712, 0.838575480617712]